In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.mode.chained_assignment = None #aby nebylo zbytečné varování

In [3]:
soubor = pd.read_csv('years/all_years.csv',  encoding='utf-8')
soubor.tail(3)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
2097293,2018.03,LEKARNA,V12,72175.0,ZINCUM METALLICUM,3CH-200CH GRA 1X4G,BNM,F,D,300.0,58.798867,17639.66,NaN,NaN,R
2097294,2018.03,LEKARNA,V12,72175.0,ZINCUM METALLICUM,3CH-200CH GRA 1X4G,BNM,F,V,-10.0,61.625000,-616.25,NaN,NaN,R
2097295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Pro nás je důležité, aby ve sloupci "Období"  nebyly prázdné hodnoty. Prověříme! 

In [4]:
soubor[soubor['Období'].isnull()].head(3)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
1636443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1660562,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1684857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Kolik máme prázdných řádků?

In [5]:
soubor[soubor['Období'].isnull()].shape[0]

20

#### Ty řádky odstraníme pomocí funkce "dropna", ale napřed prověřime, kolik zůstane řádků po odstranění:

In [6]:
len(soubor.dropna(axis=0, how='all'))

2097276

In [7]:
soubor.shape[0]

2097296

#### Rozdíl je 20 řádků. To sedí! Vytvoříme nový soubor, aby se nám nic neztratilo.

In [8]:
soubor1 = soubor.dropna(axis=0, how='all') #0 označuje 'index' a 'all' nám odstraní ty řádky, které jsou prázdné 'NaN'

In [9]:
soubor1[soubor1['Období'].isnull()].shape[0]

0

In [10]:
soubor1.shape

(2097276, 15)

In [11]:
# Zjistit duplikátní hodnoty
soubor1[soubor1.duplicated(keep = 'first')]

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje


#### Bohužel sloupce "Období" v EXCEL souborech nebyli zapsané ve formátu "Datum". Proto při nahrávání a spojení tabulek jsou tyto sloupce ve formatu "Float" - desetinné číslo. 

##### Ale my chceme místo "YYYY.MM" mít "YYYY-MM-DD", kde DD bude vždy 01 (první den měsíce). Proto napřed z našeho formátu "YYYY.MM" uděláme jiný formát "YYYYMM01" tak, že vynásobíme 10000 a přidáme 1. Pak převedeme formát na celé číslo.

In [12]:
soubor1['Období'] = ((soubor1['Období'])*10000+1).astype(np.int)
soubor1.head(1)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
0,20110101,LEKARNA,A01A,53907.0,SEPTOMIXINE,STM PST 1X7.5GM,SPH,F,D,60.0,999.65,59979.0,NaN,NaN,R


In [13]:
soubor1['Období'].dtype

dtype('int32')

##### Super, podařilo se. Teď můžeme naše celá čísla převést na formát "datum" a podívame se na výsledek:

In [14]:
soubor1['Období'] = pd.to_datetime(soubor1['Období'], format='%Y%m%d', errors='ignore')
soubor1.tail(3)

,Období,Typ odběratele,ATC7,Kód SÚKL,Název přípravku,Doplněk názvu,Držitel registračního rozhodnutí,Země,Typ pohybu,Počet balení/M,Cena za balení bez obch. přirážky a DPH,Celkem finance za všechna balení bez obch. přirážky a DPH,Počet definovaných denních dávek/balení,Počet denních definovaných dávek/balení celkem,Způsob výdeje
2097292,2018-03-01,DISTRIBUTOR_CR,V12,72175.0,ZINCUM METALLICUM,3CH-200CH GRA 1X4G,BNM,F,D,448.0,NaN,NaN,NaN,NaN,R
2097293,2018-03-01,LEKARNA,V12,72175.0,ZINCUM METALLICUM,3CH-200CH GRA 1X4G,BNM,F,D,300.0,58.798867,17639.66,NaN,NaN,R
2097294,2018-03-01,LEKARNA,V12,72175.0,ZINCUM METALLICUM,3CH-200CH GRA 1X4G,BNM,F,V,-10.0,61.625000,-616.25,NaN,NaN,R


#### Uložíme výsledek do csv souboru s názvem 'clean_data_years'

In [15]:
soubor1.to_csv('clean_data_years.csv', encoding = 'utf-8')

### ATC jsem vybrala tak, že jsem si našla některé léky prodejné v lékárných v kategirii chřipka a nachlazení ( rýma, bolest těla, kašel, horečka). Pokud se jeden přípravek objevil v kontrétní ATC skupině, tak jsem rovnou celou příslušnou ATC skupinu zařadila do našeho výběru.


In [16]:
soubor1['Typ odběratele'].unique() #Zkontrolujeme, jestli se během let nezměnilo názvosloví ve sloupci 'Typ odběratele'

array(['LEKARNA', 'DISTRIBUTOR_CR', 'VETERINARNI_LEKAR', 'LEKAR',
       'DISTRIBUTOR_EU', 'ZDRAVOTNICKE_ZARIZENI', 'OBCHODNI_ZASTUPCE',
       'ZAHRANICI', 'PRODEJCE_VLP', 'DISTRIBUTOR_MIMO_EU',
       'OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI', 'HYGIENICKA_STANICE',
       'TRANSFUZNI_SLUZBA'], dtype=object)

In [17]:
# Kromě vybraných ATC skupin pro naši analýzu je ještě třeba vybrat ze sloupce 'Typ odběratele' odpovídající odběratele, u kterých dochází k výdeji přímo pacientům.
clean_data_years_filter = soubor1[soubor1["Typ odběratele"].isin(['LEKARNA', 'VETERINARNI_LEKAR', 'PRODEJCE_VLP','HYGIENICKA_STANICE',
                                          'ZDRAVOTNICKE_ZARIZENI','LEKAR','OSOBA_POSKYTUJICI_ZDRAVOTNI_PECI'])]
clean_data_years_filter.shape

(1350626, 15)

In [18]:
# Předešlou podnímku si zkopírujeme do buňky a použijeme ji na výběr daných skupin ATC ze souboru clean_data_years.csv
clean_data_years_filter2= clean_data_years_filter [clean_data_years_filter["ATC7"].isin(['M01AE','M01AE01','M01AE02','M01AE03','M01AE04','M01AE05','M01AE06','M01AE07','M01AE08',
'M01AE09','M01AE10','M01AE11','M01AE12','M01AE13','M01AE14','M01AE15','M01AE16','M01AE17','M01AE18','M01AE51','M01AE52',
'M01AE53','M01AE56','N02B','N02BA','N02BA01','N02BA02','N02BA03','N02BA04','N02BA05','N02BA06','N02BA07','N02BA08',
'N02BA09','N02BA10','N02BA11','N02BA12','N02BA14','N02BA15','N02BA16','N02BA51','N02BA55','N02BA57','N02BA59','N02BA65',
'N02BA71','N02BA75','N02BA77','N02BA79','N02BE','N02BE01','N02BE03','N02BE04','N02BE05','N02BE51','N02BE53','N02BE54',
'N02BE71','N02BE73','N02BE74','R','R01','R01A','R01AA','R01AA02','R01AA03','R01AA04','R01AA05','R01AA06','R01AA07','R01AA08',
'R01AA09','R01AA10','R01AA11','R01AA12','R01AA13','R01AA14','R01AA15','R01AB','R01AB01','R01AB02','R01AB03','R01AB05',
'R01AB06','R01AB07','R01AB08','R01AD','R01AD01','R01AD02','R01AD03','R01AD04','R01AD05','R01AD06','R01AD07','R01AD08',
'R01AD09','R01AD11','R01AD12','R01AD13','R01AD52','R01AD53','R01AD57','R01AD58','R01AD60','R05','R05C','R05CA','R05CA01',
'R05CA02','R05CA03','R05CA04','R05CA05','R05CA06','R05CA07','R05CA08','R05CA09','R05CA10','R05CA11','R05CA12','R05CA13',
'R05CB','R05CB01','R05CB02','R05CB03','R05CB04','R05CB05','R05CB06','R05CB07','R05CB08','R05CB09','R05CB10','R05CB11',
'R05CB13','R05CB14','R05CB15','R05CB16','R05D','R05DA','R05DA01','R05DA03','R05DA04','R05DA05','R05DA06','R05DA07','R05DA08',
'R05DA09','R05DA10','R05DA11','R05DA12','R05DA20','R05DB','R05DB01','R05DB02','R05DB03','R05DB04','R05DB05','R05DB07',
'R05DB09','R05DB10','R05DB11','R05DB12','R05DB13','R05DB14','R05DB15','R05DB16','R05DB17','R05DB18','R05DB19','R05DB20',
'R05DB21','R05DB22','R05DB23','R05DB24','R05DB25','R05DB26','R05DB27','R05DB28','R05F','R05FB','R05FB01','R05FB02','R05X',
'V11'])]
clean_data_years_filter2.shape

(87958, 15)

Vytvořily jsme konečný soubor, který obsahuje distribuci LP za definované období a je vyfiltrován podle potřebných ATC a Typu odběratele.

In [19]:
clean_data_years_filter2.to_csv('clean_data_years_filter.csv', index = False, encoding = 'utf-8')